## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-14-08-29-10 +0000,bbc,German backpacker lost in outback 'beyond grat...,https://www.bbc.com/news/articles/cvg8zpej236o
1,2025-07-14-08-01-26 +0000,bbc,Don't vilify Air India crash crew: Indian pilo...,https://www.bbc.com/news/articles/cddzp0g5l25o
2,2025-07-14-08-00-00 +0000,cbc,Frostbite and fear: Inside a journey into Cana...,https://www.cbc.ca/news/canada/refugee-canada-...
3,2025-07-14-07-44-37 +0000,bbc,World's oldest president to seek eighth term,https://www.bbc.com/news/articles/ckglpnk0kqko
4,2025-07-14-06-32-22 +0000,bbc,Housing-starved Hong Kong turns Covid quaranti...,https://www.bbc.com/news/articles/cd97pxd49v2o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2321/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
85,trump,5
25,president,3
134,south,3
126,killed,3
20,canada,2


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
8,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...,25
5,2025-07-14-06-08-00 +0000,wsj,"China’s exports grew at a faster clip in June,...",https://www.wsj.com/economy/trade/chinas-expor...,18
18,2025-07-13-20-15-07 +0000,nyt,South African President Opens Corruption Inqui...,https://www.nytimes.com/2025/07/13/world/afric...,15
15,2025-07-13-22-20-06 +0000,bbc,Two women killed at Kentucky church as gunman ...,https://www.bbc.com/news/articles/cm2l5jn6254o,14
13,2025-07-13-23-01-45 +0000,bbc,Trump will be hosted by King at Windsor during...,https://www.bbc.com/news/articles/c4g25ne7gw6o,13


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
8,25,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...
5,16,2025-07-14-06-08-00 +0000,wsj,"China’s exports grew at a faster clip in June,...",https://www.wsj.com/economy/trade/chinas-expor...
18,15,2025-07-13-20-15-07 +0000,nyt,South African President Opens Corruption Inqui...,https://www.nytimes.com/2025/07/13/world/afric...
15,12,2025-07-13-22-20-06 +0000,bbc,Two women killed at Kentucky church as gunman ...,https://www.bbc.com/news/articles/cm2l5jn6254o
6,11,2025-07-14-05-46-00 +0000,bbc,Aviation authority says Boeing fuel switches a...,https://www.bbc.com/news/articles/ce9xpgnx3vdo
25,11,2025-07-13-15-20-08 +0000,bbc,Texas camp buildings were removed from map sho...,https://www.bbc.com/news/articles/ce8zjk5yx8wo
24,10,2025-07-13-16-13-14 +0000,nyt,"In Canada’s Northern Outposts, Rusting Relics ...",https://www.nytimes.com/2025/07/13/world/ameri...
22,10,2025-07-13-16-51-25 +0000,bbc,Former Nigerian leader Muhammadu Buhari dies a...,https://www.bbc.com/news/articles/c89781qxnzdo
21,10,2025-07-13-16-53-08 +0000,cbc,Anand says Indo-Pacific strategy will have eco...,https://www.cbc.ca/news/politics/indo-pacific-...
4,9,2025-07-14-06-32-22 +0000,bbc,Housing-starved Hong Kong turns Covid quaranti...,https://www.bbc.com/news/articles/cd97pxd49v2o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
